# Various Problem Solving algorithms
1. Tree Search
2. Graph Search
3. Breath First Search using graph
4. Depth First Search

Reference: 
a) The AIMA book
b) Code at https://github.com/aimacode/aima-python

In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation
from time import sleep

import math
import utils

# Switch backend
plt.switch_backend('Qt5Agg')
mpl.get_backend()



'Qt5Agg'

## Section 1: Common infrastructure

### The map of Romania

In [2]:

romania_map = dict(
    Arad=dict(Zerind=75, Sibiu=140, Timisoara=118),
    Bucharest=dict(Urziceni=85, Pitesti=101, Giurgiu=90, Fagaras=211),
    Craiova=dict(Drobeta=120, Rimnicu=146, Pitesti=138),
    Drobeta=dict(Mehadia=75),
    Eforie=dict(Hirsova=86),
    Fagaras=dict(Sibiu=99),
    Hirsova=dict(Urziceni=98),
    Iasi=dict(Vaslui=92, Neamt=87),
    Lugoj=dict(Timisoara=111, Mehadia=70),
    Oradea=dict(Zerind=71, Sibiu=151),
    Pitesti=dict(Rimnicu=97),
    Rimnicu=dict(Sibiu=80),
    Urziceni=dict(Vaslui=142))

romania_map_locations = dict(
    Arad=(91, 492), Bucharest=(400, 327), Craiova=(253, 288),
    Drobeta=(165, 299), Eforie=(562, 293), Fagaras=(305, 449),
    Giurgiu=(375, 270), Hirsova=(534, 350), Iasi=(473, 506),
    Lugoj=(165, 379), Mehadia=(168, 339), Neamt=(406, 537),
    Oradea=(131, 571), Pitesti=(320, 368), Rimnicu=(233, 410),
    Sibiu=(207, 457), Timisoara=(94, 410), Urziceni=(456, 350),
    Vaslui=(509, 444), Zerind=(108, 531))


### The Graph class
Which encapsulates the above map (and can plot it too)

In [3]:
class Graph(object):
    
    def __init__(self, dictionary=None, locations=None, directed=True):
        self.dictionary = dictionary or {}
        self.directed = directed
        self.locations = locations
        if not self.directed:
            self.make_undirected()
        
            
    def make_undirected(self):
        for city in list(self.dictionary.keys()):
            for (target, cost) in self.dictionary[city].items():
                self.connect_nodes(city, target, cost)
    
    def connect_nodes(self, a, b, cost):
        self.connect_nodes_helper(a, b, cost)
        if not self.directed:
            self.connect_nodes_helper(b, a, cost)
    
    def connect_nodes_helper(self, a, b, cost):
        self.dictionary.setdefault(a, {})[b] = cost
        
    def get_nodes(self):
        return list(self.dictionary.keys())
    
    def get_connections(self, a, b=None):
        connections = self.dictionary.setdefault(a, {})
        if b is None:
            return connections
        else:
            return connections[b]
        
        
        

### Plot class can plog a Graph object

In [29]:

class Plot(object):
    def __init__(self):
        self.fig = plt.figure(figsize=(15, 10))
        self.sp = self.fig.add_subplot(111)
        
        
    def plot(self, graph, thetitle = "The Plot", solution = None, solution_color = 'red', tries=None):
        if g.locations is None:
            print("No locations given. Cannot plot.")
            return

        x_max = -float('inf')
        x_min = float('inf')

        y_max = -float('inf')
        y_min = float('inf')

        x = []
        y = []
        names = []

        for city in g.locations:
            if g.locations[city][0] < x_min:
                x_min = g.locations[city][0]
            if g.locations[city][0] > x_max:
                x_max = g.locations[city][0]

            if g.locations[city][1] < y_min:
                y_min = g.locations[city][1]
            if g.locations[city][1] > y_max:
                y_max = g.locations[city][1]

            x.append(g.locations[city][0])
            y.append(g.locations[city][1])
            names.append(city)


        if x_max > y_max:
            the_max = x_max
        else:
            the_max = y_max

        border = 50

        self.sp.set(xlim=[x_min-border, x_max+border], ylim=[y_min-border, y_max+border],
               title=thetitle, ylabel='Y-Axis', xlabel='X-Axis')

        self.sp.scatter(x, y, color='tan', marker='o', s=100)

        for i, name in enumerate(names):
            self.sp.annotate(name, (x[i] + 4,y[i] + 4))

        for city in g.dictionary:
            for target in g.dictionary[city]:
                x1 = g.locations[city][0]
                y1 = g.locations[city][1]
                x2 = g.locations[target][0]
                y2 = g.locations[target][1]
                lines, = self.sp.plot([x1, x2], [y1, y2], color='gray')
                lines.set_linestyle(':')
                distance = str(g.dictionary[city][target])
                xa = min(x1, x2)
                xb = max(x1, x2)
                ya = min(y1, y2)
                yb = max(y1, y2)
                self.sp.annotate(distance, (xa + (xb-xa)/2, (ya + (yb-ya)/2)), color='teal')


        if solution:
            previous = None
            for step in solution:
                if previous is not None:
                    x1 = g.locations[previous.state][0]
                    y1 = g.locations[previous.state][1]
                    x2 = g.locations[step.state][0]
                    y2 = g.locations[step.state][1]
                    lines, = self.sp.plot([x1, x2], [y1, y2], color=solution_color)
                previous = step

        #plt.tight_layout(pad=-20)
        plt.grid()
        #plt.plot()
        #plt.show()
        
        
def animate(i, itera, target, cost = None):

    c = 'red'
    if i == len(itera) - 1:
        c = 'green'
        
    p.sp.clear()
    p.plot(g, title, itera[i], solution_color=c)
    sites = itera[i]
    
    x = romania_map_locations[target][0]
    y = romania_map_locations[target][1]
    s = p.sp.scatter(x, y, color='lime', marker='o', s=100)

    for site in sites:
        x = romania_map_locations[site.state][0]
        y = romania_map_locations[site.state][1]
        s = p.sp.scatter(x, y, color=c, marker='o', s=100)
    
    if cost:    
        x = romania_map_locations[sites[-1].state][0]
        y = romania_map_locations[sites[-1].state][1]
        p.sp.text(x, y, str(cost[i]), fontsize=24)
        
    p.sp.text(200, 550, "Iteration: " + str(i+1), fontsize=24)
    
    return s

Test it out

In [5]:

g = Graph(romania_map, romania_map_locations, False)
print(g.get_connections('Arad', 'Timisoara'))
print(g.get_connections('Urziceni'))

# Test Graph and animation

#p = Plot()
#p.plot(g)

#def animate(i):
#    s = p.sp.scatter([200+i, 300+i], [400, 500],  color='red', marker='o', s=100)
#    return s

#anim = animation.FuncAnimation(p.fig, animate, frames=10, interval=1000, blit=False)
#plt.show()




118
{'Hirsova': 98, 'Bucharest': 85, 'Vaslui': 142}


### The GraphProblem class
Which encapsulates the problem (initial state, goal state, actions, results, path cost)

In [6]:
class GraphProblem(object):
    """The Problem Definition"""
    def __init__(self, graph, initial_state, goal_state):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.graph = graph
      
    
    def actions(self, state):
        return list(self.graph.get_connections(state))
    
    
    def results(self, state, action):
        if action in self.graph.get_connections(state):
            return action
        else:
            return None
        
        
    def goal_test(self, state):
        return self.goal_state == state
    
    
    def path_cost(self, c, state1, state2, action):
        return c + self.graph.get_connections(state1, state2)
    
    
    def h(self, node):
        locs = getattr(self.graph, 'locations', None)
        if locs:
            return int(distance(locs[node.state], locs[self.goal_state]))
        else:
            return infinity
        

Test it out

In [7]:
g = Graph(romania_map, romania_map_locations, False)
gp = GraphProblem(g, 'Arad', 'Bucharest')
print (gp.actions('Urziceni'))
print (gp.results('Sibiu','Arad'))
print (gp.path_cost(10, 'Arad', 'Timisoara', 'Timisoara'))

['Hirsova', 'Bucharest', 'Vaslui']
Arad
128


### The Node class
Which represents the frontiers and explored nodes. Maintains the path travelled to reach it. Path cost. Parent.

In [8]:
class Node(object):
    
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        
        if self.parent:
            self.depth = self.parent.depth + 1
            
        
    def __repr__(self):
        return "{0}".format(self.state)
    
    
    def __lt__(self, node):
        return self.state < node.state
    
    
    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state
    
    
    def __hash__(self):
        return hash(self.state)
        
    
    def expand(self, problem):
        return [self.child_node(problem, action)
            for action in problem.actions(self.state)]
    
    
    def child_node(self, problem, action):
        next_state = problem.results(self.state, action)
        return Node(next_state, self, action, problem.path_cost(self.path_cost, self.state, next_state, action))
    
    
    def path(self):
        node = self
        path_back = []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))
    
    
    def solution(self):
        return [node.action for node in self.path()[1:]]

## Tree Search

In [9]:
###########################################################
# Tree search algorithm
###########################################################
def tree_search(problem, frontier):
    
    frontier.append(Node(problem.initial_state))
    iterations = []
    
    # Algorithm starts
    while frontier:

        path = frontier.pop()
        iterations.append(path.path())

        if problem.goal_test(path.state):
            return True, iterations, path
        
        paths = path.expand(problem)
        for p in paths:
            frontier.append(p)
        
    return False, None, None

In [13]:
###########################################################
# Execute BFS Tree Search and plot the graph and solution
###########################################################
title = "Default BFS Tree Search"
gp = GraphProblem(g, 'Arad', 'Neamt')
result, iterations, path = tree_search(gp, utils.FIFOQueue())
print(result)
print(path.path())
print("Tries:")
print("------------------------------------------")
for i in iterations:
    print(i)

p = Plot()
p.plot(g, title)
p.sp.set_title(title)

anim = animation.FuncAnimation(p.fig, animate, frames=len(iterations), interval=100, blit=False, fargs=[iterations, gp.goal_state])
plt.show()

True
[Arad, Sibiu, Fagaras, Bucharest, Urziceni, Vaslui, Iasi, Neamt]
Tries:
------------------------------------------
[Arad]
[Arad, Sibiu]
[Arad, Timisoara]
[Arad, Zerind]
[Arad, Sibiu, Rimnicu]
[Arad, Sibiu, Arad]
[Arad, Sibiu, Oradea]
[Arad, Sibiu, Fagaras]
[Arad, Timisoara, Arad]
[Arad, Timisoara, Lugoj]
[Arad, Zerind, Arad]
[Arad, Zerind, Oradea]
[Arad, Sibiu, Rimnicu, Craiova]
[Arad, Sibiu, Rimnicu, Sibiu]
[Arad, Sibiu, Rimnicu, Pitesti]
[Arad, Sibiu, Arad, Sibiu]
[Arad, Sibiu, Arad, Timisoara]
[Arad, Sibiu, Arad, Zerind]
[Arad, Sibiu, Oradea, Sibiu]
[Arad, Sibiu, Oradea, Zerind]
[Arad, Sibiu, Fagaras, Sibiu]
[Arad, Sibiu, Fagaras, Bucharest]
[Arad, Timisoara, Arad, Sibiu]
[Arad, Timisoara, Arad, Timisoara]
[Arad, Timisoara, Arad, Zerind]
[Arad, Timisoara, Lugoj, Timisoara]
[Arad, Timisoara, Lugoj, Mehadia]
[Arad, Zerind, Arad, Sibiu]
[Arad, Zerind, Arad, Timisoara]
[Arad, Zerind, Arad, Zerind]
[Arad, Zerind, Oradea, Sibiu]
[Arad, Zerind, Oradea, Zerind]
[Arad, Sibiu, Rimnicu, C

In [ ]:
###########################################################
# Execute DFS Tree Search and plot the graph and solution
###########################################################

# !! WARNING !!
# Gets into infinite loop, because earlier visited nodes are unchecked
# Hence disabling execution
"""
title = "Default DFS Tree Search"
gp = GraphProblem(g, 'Arad', 'Sibiu')
result, iterations, path = tree_search(gp, utils.Stack())
print(result)
print(path.path())
print("Tries:")
print("------------------------------------------")
for i in iterations:
    print(i)

p = Plot()
p.plot(g, title)
p.sp.set_title(title)

def animate(i):

    c= 'red'
    if i == len(iterations) - 1:
        c = 'green'
        
    p.sp.clear()
    p.plot(g, title, iterations[i], solution_color=c)
    sites = iterations[i]
    for site in sites:
        x = romania_map_locations[site.state][0]
        y = romania_map_locations[site.state][1]
        s = p.sp.scatter(x, y, color=c, marker='o', s=100)
    return s


anim = animation.FuncAnimation(p.fig, animate, frames=len(iterations), interval=1000, blit=False)
plt.show()
"""

## Graph Search

In [14]:

def graph_search(problem, frontier):
    
    frontier.append(Node(problem.initial_state))
    explored = set()
    iterations = []
    
    # Algorithm starts
    while  frontier:
        path = frontier.pop()
        explored.add(path.state)
        iterations.append(path.path())
        
        if problem.goal_test(path.state):
            return True, iterations, path
        
        paths = path.expand(problem)
        for p in paths:
            if p.state not in explored and p not in frontier:
                frontier.append(p)
        
    return False, None, None


In [15]:
###########################################################
# Execute BFS Graph Search and plot the graph and solution
###########################################################
title = "BFS Graph Search"
gp = GraphProblem(g, 'Arad', 'Neamt')
result, iterations, path = graph_search(gp, utils.FIFOQueue())
print(result)
print(path.path())
print("Tries:")
print("------------------------------------------")
for i in iterations:
    print(i)

title = "BFS Graph Search"
p = Plot()
p.plot(g, title)
p.sp.set_title(title)



anim = animation.FuncAnimation(p.fig, animate, frames=len(iterations), interval=1000, blit=False, fargs=[iterations, gp.goal_state])
plt.show()


True
[Arad, Sibiu, Fagaras, Bucharest, Urziceni, Vaslui, Iasi, Neamt]
Tries:
------------------------------------------
[Arad]
[Arad, Sibiu]
[Arad, Timisoara]
[Arad, Zerind]
[Arad, Sibiu, Rimnicu]
[Arad, Sibiu, Oradea]
[Arad, Sibiu, Fagaras]
[Arad, Timisoara, Lugoj]
[Arad, Sibiu, Rimnicu, Craiova]
[Arad, Sibiu, Rimnicu, Pitesti]
[Arad, Sibiu, Fagaras, Bucharest]
[Arad, Timisoara, Lugoj, Mehadia]
[Arad, Sibiu, Rimnicu, Craiova, Drobeta]
[Arad, Sibiu, Fagaras, Bucharest, Giurgiu]
[Arad, Sibiu, Fagaras, Bucharest, Urziceni]
[Arad, Sibiu, Fagaras, Bucharest, Urziceni, Hirsova]
[Arad, Sibiu, Fagaras, Bucharest, Urziceni, Vaslui]
[Arad, Sibiu, Fagaras, Bucharest, Urziceni, Hirsova, Eforie]
[Arad, Sibiu, Fagaras, Bucharest, Urziceni, Vaslui, Iasi]
[Arad, Sibiu, Fagaras, Bucharest, Urziceni, Vaslui, Iasi, Neamt]


In [16]:
###########################################################
# Execute DFS Graph Search and plot the graph and solution
###########################################################
title = "DFS Graph Search"
gp = GraphProblem(g, 'Arad', 'Neamt')
result, iterations, path = graph_search(gp, utils.Stack())
print(result)
print(path.path())
print("Tries:")
print("------------------------------------------")
for i in iterations:
    print(i)

title = "DFS Graph Search"
p = Plot()
p.plot(g, title)
p.sp.set_title(title)

anim = animation.FuncAnimation(p.fig, animate, frames=len(iterations), interval=1000, blit=False, fargs=[iterations, gp.goal_state])
plt.show()


True
[Arad, Timisoara, Lugoj, Mehadia, Drobeta, Craiova, Pitesti, Bucharest, Urziceni, Vaslui, Iasi, Neamt]
Tries:
------------------------------------------
[Arad]
[Arad, Zerind]
[Arad, Zerind, Oradea]
[Arad, Timisoara]
[Arad, Timisoara, Lugoj]
[Arad, Timisoara, Lugoj, Mehadia]
[Arad, Timisoara, Lugoj, Mehadia, Drobeta]
[Arad, Timisoara, Lugoj, Mehadia, Drobeta, Craiova]
[Arad, Timisoara, Lugoj, Mehadia, Drobeta, Craiova, Pitesti]
[Arad, Timisoara, Lugoj, Mehadia, Drobeta, Craiova, Pitesti, Bucharest]
[Arad, Timisoara, Lugoj, Mehadia, Drobeta, Craiova, Pitesti, Bucharest, Fagaras]
[Arad, Timisoara, Lugoj, Mehadia, Drobeta, Craiova, Pitesti, Bucharest, Urziceni]
[Arad, Timisoara, Lugoj, Mehadia, Drobeta, Craiova, Pitesti, Bucharest, Urziceni, Vaslui]
[Arad, Timisoara, Lugoj, Mehadia, Drobeta, Craiova, Pitesti, Bucharest, Urziceni, Vaslui, Iasi]
[Arad, Timisoara, Lugoj, Mehadia, Drobeta, Craiova, Pitesti, Bucharest, Urziceni, Vaslui, Iasi, Neamt]


## Depth Limit Search

In [17]:

def depth_limit_search(problem, limit=10):
    
    visited = []
    
    def recursive_dls(thisnode, problem, iterations, limit):

        iterations.append(thisnode.path())
        visited.append(thisnode.state)
       
        if problem.goal_test(thisnode.state):
            return True, iterations, thisnode
        
        if limit == 0:
            return False, iterations, thisnode
            
        nodes = thisnode.expand(problem)
        for node in nodes:
            if node.state not in visited:
                result, iterations, path = recursive_dls(node, problem, iterations, limit - 1)
                if result:
                    return result, iterations, path

                
        return False, iterations, node.path()

    iterations = []
    return recursive_dls(Node(problem.initial_state), problem, iterations, limit)
    
    


In [31]:
gp = GraphProblem(g, 'Arad', 'Giurgiu')
result, iterations, node = depth_limit_search(gp, 4)
print("--------")
print(node.path())

title = "Depth Limit Search"
p = Plot()
p.plot(g, title)
p.sp.set_title(title)

anim = animation.FuncAnimation(p.fig, animate, frames=len(iterations), interval=500, blit=False, fargs=[iterations, gp.goal_state])
plt.show()


--------
[Arad, Sibiu, Fagaras, Bucharest, Giurgiu]


## Iterative Deepening Search

In [21]:
def iterative_deepening_search(problem):
    iterations = []
    for i in range(12):
        result, it, node = depth_limit_search(gp, i)
        iterations.extend(it)
        if result:
            return result, iterations, node


In [22]:
gp = GraphProblem(g, 'Arad', 'Drobeta')
result, iterations, node = iterative_deepening_search(gp)
print("--------")
print(node.path())

title = "Iterative Deepening Search"
p = Plot()
p.plot(g, title)
p.sp.set_title(title)

anim = animation.FuncAnimation(p.fig, animate, frames=len(iterations), interval=500, blit=False, fargs=[iterations, gp.goal_state])
plt.show()

--------
[Arad, Sibiu, Rimnicu, Craiova, Drobeta]


## Best First Path Search (a group of algos)

In [23]:
def distance(a, b):
    """The distance between two (x, y) points."""
    return math.hypot((a[0] - b[0]), (a[1] - b[1]))


def best_first_search(problem, cost_fn):
    
    frontier = utils.PriorityQueue(min, cost_fn)
    frontier.append(Node(problem.initial_state))
    explored = set()
    iterations = []
    path_cost = []
    
    # Algorithm starts
    while frontier:
        path = frontier.pop()
        explored.add(path.state)
        iterations.append(path.path())
        path_cost.append(path.path_cost)
                    
        if problem.goal_test(path.state):
            return True, iterations, path, path_cost
        
        paths = path.expand(problem)
        for child in paths:
            if child.state not in explored and child not in frontier:
                frontier.append(child)
            elif child in frontier:
                incumbent = frontier[child]
                if cost_fn(child) < cost_fn(incumbent):
                    del frontier[incumbent]
                    frontier.append(child)
                
        
    return False, None, None, None

def uniform_cost_search(problem):
    return best_first_search(problem, lambda node: node.path_cost)


def astar_search(problem):
    h = problem.h
    return best_first_search(problem, lambda node: node.path_cost + h(node))

def greedy_best_first_graph_search(problem):
    h = problem.h
    return best_first_search(problem, lambda node: h(node))

### Uniform cost search

In [24]:
gp = GraphProblem(g, 'Arad', 'Bucharest')
result, iterations, node, path_cost = uniform_cost_search(gp)
print("--------")
print(node.path())

for i in iterations:
    print(i)

title = "Uniform Cost Search"
p = Plot()
p.plot(g, title)
p.sp.set_title(title)

anim = animation.FuncAnimation(p.fig, animate, frames=len(iterations), interval=1000, blit=False, fargs=[iterations, gp.goal_state, path_cost])
plt.show()


--------
[Arad, Sibiu, Rimnicu, Pitesti, Bucharest]
[Arad]
[Arad, Zerind]
[Arad, Timisoara]
[Arad, Sibiu]
[Arad, Zerind, Oradea]
[Arad, Sibiu, Rimnicu]
[Arad, Timisoara, Lugoj]
[Arad, Sibiu, Fagaras]
[Arad, Timisoara, Lugoj, Mehadia]
[Arad, Sibiu, Rimnicu, Pitesti]
[Arad, Sibiu, Rimnicu, Craiova]
[Arad, Timisoara, Lugoj, Mehadia, Drobeta]
[Arad, Sibiu, Rimnicu, Pitesti, Bucharest]


### A* Search


In [30]:
gp = GraphProblem(g, 'Arad', 'Bucharest')
result, iterations, node, path_cost = astar_search(gp)
print("--------")
print(node.path())
for i in iterations:
    print(i)

title = "A* Search"
p = Plot()
p.plot(g, title)
p.sp.set_title(title)

anim = animation.FuncAnimation(p.fig, animate, frames=len(iterations), interval=1000, blit=False, fargs=[iterations, gp.goal_state, path_cost])
plt.show()

--------
[Arad, Sibiu, Rimnicu, Pitesti, Bucharest]
[Arad]
[Arad, Sibiu]
[Arad, Sibiu, Fagaras]
[Arad, Sibiu, Rimnicu]
[Arad, Sibiu, Rimnicu, Pitesti]
[Arad, Sibiu, Rimnicu, Pitesti, Bucharest]


In [32]:
gp = GraphProblem(g, 'Arad', 'Giurgiu')
result, iterations, node, path_cost = greedy_best_first_graph_search(gp)
print("--------")
print(node.path())
for i in iterations:
    print(i)

title = "Greedy Best First Search"
p = Plot()
p.plot(g, title)
p.sp.set_title(title)

anim = animation.FuncAnimation(p.fig, animate, frames=len(iterations), interval=1000, blit=False, fargs=[iterations, gp.goal_state, path_cost])
plt.show()

--------
[Arad, Sibiu, Fagaras, Bucharest, Giurgiu]
[Arad]
[Arad, Sibiu]
[Arad, Sibiu, Fagaras]
[Arad, Sibiu, Fagaras, Bucharest]
[Arad, Sibiu, Fagaras, Bucharest, Giurgiu]
